In [1]:
import requests, httpx
import json
from urllib.parse import urlencode, quote, urlsplit, parse_qs, parse_qsl, unquote, unquote_plus
from barely_json import parse
import jmespath
headers = {
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9,ar;q=0.8',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
    'x-asbd-id': '129477',
    'x-ig-app-id': '936619743392459',
    'cookie': '''csrftoken=cpBvgDRGLIQ4j3RgScGvH24w9R6cQffX; ds_user_id=18519728814; ps_n=1; ps_l=1; mid=Zl0FuQALAAHWLneH1O51KnPmZbQO; ig_did=8F185B9B-0018-4D2C-9C8B-5861E8826960; dpr=0.8640000224113464; datr=fid7ZoVfi9JAdPRsF4AUt5H9; shbid="19508\05418519728814\0541751227322:01f71e1583bc74deb0cf35aadeef14cbf22516af31cc981c85d6a595394c0fc9c9797cbb"; shbts="1719691322\05418519728814\0541751227322:01f797a9ebf1355c2fe47a789c2fca5f528433508e5c351bbb67dc69bcb3920b918ea3d3"; sessionid=18519728814%3A8CptQEBMl6lstb%3A29%3AAYfPHfaU0TD5O2F2otwE2BteTZDIh3lS8N5qbid1QQ; rur="RVA\05418519728814\0541751409798:01f7914db0d124d7da5a7af3e8dbeca4c2fc69e72380836ba0862459726e5ae2d6f8640f"; wd=1582x268'''
    }

In [190]:
def prepare_endpoint(variables):
    '''Return an api endpoint for data about the first 12 posts on a user page'''
    json_str = json.dumps(vars_dict)
    return f"https://www.instagram.com/graphql/query/?query_hash=e769aa130647d2354c40ea6a439bfc08&variables={quote(json_str)}"

variables = {
  "id": "1067259270",
  "first": 12,
  "after": None
}
prepare_endpoint(variables)

'https://www.instagram.com/graphql/query/?query_hash=e769aa130647d2354c40ea6a439bfc08&variables=%7B%22id%22%3A%201067259270%2C%20%22first%22%3A%2012%7D'

In [125]:
def change_variables(url, first=12):
    ''' ! Use prepare_endpoint() instead ! \n
    Parse an already existing all-user posts endpoint, allows manipulating it and 
    returns new endpoint 
    '''
    url_parts = urlparse(url)
    query = url_parts.query
    variables = parse_qs(query)['variables'][0]
    vars_dict = parse(variables)
    vars_dict['first'] = first
    vars_dict['id'] = 1067259270
    json_str = json.dumps(vars_dict)
    return f"https://www.instagram.com/graphql/query/?query_hash=e769aa130647d2354c40ea6a439bfc08&variables={quote(json_str)}"
endpoint = change_variables(endpoint)

In [126]:
resp = requests.get(endpoint)
resp.status_code

200

In [185]:
# Testing comments in posts
data = resp.json()['data']['user']['edge_owner_to_timeline_media']['edges'][0]['node']
jmespath.search('''{
    shortcode: shortcode,
    comments: edge_media_to_comment.edges[].node[].text,
    page
    }''', data)

{'shortcode': 'C8senjXOVkv',
 'comments': ['👏', 'Love her!', 'Fix my business profile!!']}

In [178]:
# Check keys for each post
resp.json()['data']['user']['edge_owner_to_timeline_media']['edges'][0]['node'].keys()

dict_keys(['__typename', 'id', 'dimensions', 'display_url', 'display_resources', 'is_video', 'tracking_token', 'edge_media_to_tagged_user', 'accessibility_caption', 'edge_media_to_caption', 'shortcode', 'edge_media_to_comment', 'edge_media_to_sponsor_user', 'comments_disabled', 'taken_at_timestamp', 'edge_media_preview_like', 'gating_info', 'fact_check_overall_rating', 'fact_check_information', 'media_preview', 'owner', 'location', 'viewer_has_liked', 'viewer_has_saved', 'viewer_has_saved_to_collection', 'viewer_in_photo_of_you', 'viewer_can_reshare', 'thumbnail_src', 'thumbnail_resources'])

In [180]:
# Check if changing returned posts number will break the api (seems like it's working)
len(resp.json()['data']['user']['edge_owner_to_timeline_media']['edges'])

20